In [152]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
 
from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

plt.style.use('seaborn') #стиль отрисовки seaborn
%matplotlib inline

C:\Users\kot\AppData\Local\Temp\ipykernel_43344\598800705.py:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn') #стиль отрисовки seaborn


In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [12]:
data = pd.read_excel('data/data_ford_price.xlsx') 
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


None

### 9. Отбор признаков: классификация методов

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

Сначала RFE обучается на изначальной выборке и происходит оценка важности каждого признака. Затем наименее важные фичи удаляются. Эта процедура рекурсивно повторяется на сокращённом наборе до тех пор, пока в конечном итоге не будет достигнуто желаемое количество признаков в выборке.

### RFE

In [427]:
from sklearn.feature_selection import RFE


y = data['price']
x = data.drop(columns='price')
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)



estimator = linear_model.LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
display(selector.get_feature_names_out())
display(X_train.columns)
display(selector.ranking_)

array(['year', 'cylinders', 'lat'], dtype=object)

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

array([1, 1, 4, 1, 3, 2])

### МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

Два наиболее популярных метода:

        выбор k лучших переменных: SelectKBest;
        выбор переменных верхнего процентиля: SelectPercentile.


In [432]:
from sklearn.feature_selection import SelectKBest, f_classif, f_regression
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [433]:
selector = SelectKBest(f_classif, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

#### Обучена регрессия на трёх столбцах,  выбранные RFE, оценено качество модели на тесте.

In [441]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.dropna(inplace = True)

y = data['price']
x = data[['year', 'cylinders', 'lat']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5143.044


#### Обучена регрессия на трёх столбцах,  выбранные SelectKBest, оценено качество модели на тесте.

In [442]:
data = pd.read_excel('data/data_ford_price.xlsx') 
data.dropna(inplace = True)

y = data['price']
x = data[['year', 'cylinders', 'odometer']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

model = linear_model.LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4660.914


Вопрос: Какой метод отбора признаков показал наилучший результат на тестовой выборке?

Ответ: Лучший резуьтат показала модель регресси, обученная на  столбцах-признаках, выбранных SelectKBes, метрика MAE у нее 4660 по сравнению с результатми работы RFE, где MAE равно 5143
